## CMIP5 Downloader (my version)
#### The standard CMIP6 downloader(cmip6_downloader.py) apprears to have some problems
#### ie. certain files are missing that are listed on the website.
#### This also needs to be adapted for CMIP5

In [121]:
import urllib.request
import json
import random
import multiprocessing
import os
import re
from datetime import datetime

In [ ]:
## Summary of needed changes
#   - maybe change limit? (probably not)
#   - hardcoded 0 in file_urls_to_download

In [122]:
number_of_processes = 50
files_to_download = multiprocessing.Manager().list()
# files_to_download = []
failed_files = multiprocessing.Manager().list()

In [102]:
# ['precipitation_flux'] == ['precipitation_flux',]

In [123]:
def get_files_to_download(url_files_search, cf_standard_name = 'precipitation_flux'):
    with urllib.request.urlopen(url_files_search) as result:
        data = json.loads(result.read().decode())
        data_docs = data['response']['docs']
        file_urls_to_download = []
        for doc_i in range(len(data_docs)):
            if cf_standard_name in data_docs[doc_i]['cf_standard_name']:
                file_urls_to_download.extend(data_docs[doc_i]['url'])
        for file_url_to_download in file_urls_to_download:
            if file_url_to_download.split('|')[2] == 'HTTPServer':
                files_to_download.append(file_url_to_download.split('|')[0])

In [124]:
variable = '&variable_id=' + 'pr' 
frequency = '&frequency=' + '3hr'
# experiment = '&experiment_id=' + 'historical'

experiment = 'historical'
    
# url = 'https://esgf-node.llnl.gov/esg-search/search/?offset=0&limit=10000&type=Dataset&replica=false&latest=true&project=CMIP6&' + variable + frequency + experiment + '&facets=mip_era%2Cactivity_id%2Cmodel_cohort%2Cproduct%2Csource_id%2Cinstitution_id%2Csource_type%2Cnominal_resolution%2Cexperiment_id%2Csub_experiment_id%2Cvariant_label%2Cgrid_label%2Ctable_id%2Cfrequency%2Crealm%2Cvariable_id%2Ccf_standard_name%2Cdata_node&format=application%2Fsolr%2Bjson'


variable = '&variable=' + 'pr' 
frequency = '&time_frequency=' + '3hr'
experiment = '&experiment=' + 'historical'
url = 'https://esgf-node.llnl.gov/esg-search/search/?offset=0&limit=1000&type=Dataset&replica=false&latest=true' + experiment + frequency + variable +'&project=CMIP5&project=TAMIP&project=EUCLIPSE&project=LUCID&project=GeoMIP&project=PMIP3&facets=project%2Cproduct%2Cinstitute%2Cmodel%2Cexperiment%2Cexperiment_family%2Ctime_frequency%2Crealm%2Ccmor_table%2Censemble%2Cvariable%2Cvariable_long_name%2Ccf_standard_name%2Cdata_node&format=application%2Fsolr%2Bjson'



# url = 'https://esgf-node.llnl.gov/esg-search/search/?offset=0&limit=10000&type=Dataset&replica=false&latest=true&project=CMIP5&' + variable + frequency + experiment + '&facets=mip_era%2Cactivity_id%2Cmodel_cohort%2Cproduct%2Csource_id%2Cinstitution_id%2Csource_type%2Cnominal_resolution%2Cexperiment_id%2Csub_experiment_id%2Cvariant_label%2Cgrid_label%2Ctable_id%2Cfrequency%2Crealm%2Cvariable_id%2Ccf_standard_name%2Cdata_node&format=application%2Fsolr%2Bjson'




In [116]:
# print(url)

In [125]:
pool_search = multiprocessing.Pool(number_of_processes)
with urllib.request.urlopen(url) as result_search:
    data = json.loads(result_search.read().decode())
    print('2- ' + str(len(data['response']['docs'])) + ' records found. Searching     for files to download inside each record...')
    for doc in data['response']['docs']:
        url_files_search = 'https://esgf-node.llnl.gov/search_files/' + doc['id'] + '/' + doc['index_node'] + '/?limit=10000&rnd=' + str(random.randint(100000, 999999))
        
#         if 'GFDL' in doc['id']:
#             print(url_files_search)
#             print('done')
        pool_search.apply_async(get_files_to_download, args=[url_files_search])
    pool_search.close()
    pool_search.join()

2- 91 records found. Searching     for files to download inside each record...


In [133]:
# url_files_search
len(list(files_to_download))

2156

In [51]:
# data
# data['response']['docs']

In [88]:
url_files_search

'https://esgf-node.llnl.gov/search_files/cmip5.output2.NASA-GISS.GISS-E2-H.historical.3hr.atmos.3hr.r6i1p2.v20160426|dataserver.nccs.nasa.gov/esgf-node.llnl.gov/?limit=10000&rnd=878567'

In [89]:
with urllib.request.urlopen(url_files_search) as result:
        data = json.loads(result.read().decode())
        data_docs = data['response']['docs']
        file_urls_to_download = []
        for doc_i in range(len(data_docs)):
            file_urls_to_download.extend(data_docs[doc_i]['url'])

In [95]:
data_docs[3]['cf_standard_name']

['precipitation_flux']

In [128]:
# list(files_to_download)
rr = list(files_to_download)

In [131]:
# rr[0].split

rr_sub = [rr_i for rr_i in rr if 'GFDL' in rr_i]

In [142]:
datetime.strptime(re.split('[. -]', rr_sub[4])[-2], '%Y%m%d%H')

datetime.datetime(1974, 12, 31, 23, 0)

In [141]:
len(re.split('[. -]', rr_sub[4])[-2])

10

In [143]:
# rr_sub

In [85]:

for doc in data['response']['docs']:
    if 'GFDL' in doc['id']:
#         print(doc)
#         print("\n")
        url_files_search = 'https://esgf-node.llnl.gov/search_files/' + doc['id'] + '/' + doc['index_node'] + '/?limit=10000&rnd=' + str(random.randint(100000, 999999))
        break
#     doc
# data['response']['docs']

In [21]:
# url_files_search = 'https://esgf-node.llnl.gov/search_files/CMIP6.CMIP.NOAA-GFDL.GFDL-CM4.historical.r1i1p1f1.3hr.pr.gr1.v20180701|esgdata.gfdl.noaa.gov/esgdata.gfdl.noaa.gov/?limit=100000&amp;rnd=688680'
url_files_search = 'https://esgf-node.llnl.gov/search_files/CMIP6.CMIP.NOAA-GFDL.GFDL-CM4.historical.r1i1p1f1.3hr.pr.gr1.v20180701|esgdata.gfdl.noaa.gov/esgdata.gfdl.noaa.gov/?limit=10000&amp;rnd=541688'

In [43]:
# with urllib.request.urlopen(url) as result_search:
#     res = get_files_to_download(url_files_search)

In [27]:
list(files_to_download)

['http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr1/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr1_196001010130-197912312230.nc',
 'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr1/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr1_196001010130-197912312230.nc']

In [119]:
# with urllib.request.urlopen(url_files_search) as result:
#     data = json.loads(result.read().decode())
#     file_urls_to_download = data['response']['docs'][0]['url']

In [41]:
# list(files_to_download)
# file_urls_to_download

In [36]:
fs = list(files_to_download)
print(len(fs))

8716


In [66]:
# 201301010130-201312312230.nc
datetime(2000, 4, 4)

In [72]:
datetime.strptime('201301010130', '%Y%m%d%H%M')

datetime.datetime(2013, 1, 1, 1, 30)

In [178]:
min_datetime = datetime(1970,1,1)
fs = list(files_to_download)
fs_sub = []
for f_i in fs:
    if len(re.split('[. -]', f_i)[-2]) == 10:
        end_date = datetime.strptime(re.split('[. -]', f_i)[-2],'%Y%m%d%H')
    else:
        end_date = datetime.strptime(re.split('[. -]', f_i)[-2],'%Y%m%d%H%M')
    if end_date >= min_datetime:
        fs_sub.append(f_i)
    

In [173]:
# fs_sub_sub = [f_i for f_i in fs_sub if 'GFDL' in f_i]

In [181]:
len(fs_sub)

1019

In [96]:
print(fs[0])
# fs[0][-25:-3]
# fs.split()

http://esgf-data2.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC6/historical/r2i1p1f1/3hr/pr/gn/v20190912/pr_3hr_MIROC6_historical_r2i1p1f1_gn_201301010130-201312312230.nc


In [97]:
re.split('[. -]', fs[0])[-2]

'201312312230'

In [37]:
fs_sub = [f_i for f_i in fs if 'GFDL' in f_i]

fs_sub

['http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr2/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr2_196001010130-197912312230.nc',
 'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr2/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr2_198001010130-199912312230.nc',
 'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr2/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr2_200001010130-201412312230.nc',
 'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr1/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr1_196001010130-197912312230.nc',
 'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr1/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr1_198001010130-199912312230.

In [40]:
# fs_sub[0] 
fs_sub[3]

'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr1/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr1_196001010130-197912312230.nc'

In [70]:
fs_sub[0] 

'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot4/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/3hr/pr/gr2/v20180701/pr_3hr_GFDL-CM4_historical_r1i1p1f1_gr2_196001010130-197912312230.nc'

In [ ]:
https://dataserver.nccs.nasa.gov/thredds/fileServer/CMIP5/NASA/GISS/historical/E2-H_historical_r6i1p2_3hr/clt_3hr_GISS-E2-H_historical_r6i1p2_195001010130-195912312230.nc

In [ ]:
'http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot/NOAA-GFDL/GFDL-ESM2M/historical/3hr/atmos/3hr/r1i1p1/v20110601/clt/clt_3hr_GFDL-ESM2M_historical_r1i1p1_1906010100-1910123123.nc'